在 Ubuntu 24.04 上使用 **Docker Compose（通过 `apt` 安装）** 管理 `timescale/timescaledb:latest-pg16` 容器，并实现：

- ✅ 使用 `apt` 安装 Docker 和 Docker Compose  
- ✅ 容器数据持久化到主机目录（外部存储映射）  
- ✅ 系统启动时自动运行容器（开机自启）

---

## ✅ 第一步：使用 `apt` 安装 Docker 和 Docker Compose

```bash
# 1. 更新系统
sudo apt update

# 2. 安装必要依赖
sudo apt install -y ca-certificates curl gnupg lsb-release

# 3. 添加 Docker 官方 GPG 密钥
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg

# 4. 添加 Docker APT 仓库
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

# 5. 安装 Docker Engine 和 Docker Compose Plugin（Ubuntu 24.04 推荐方式）
sudo apt update
sudo apt install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin

# 6. 验证安装
docker --version
docker compose version
```

> 💡 注意：Ubuntu 24.04 中 `docker-compose` 命令已由 `docker compose`（作为插件）取代，无需单独安装 Python 版本。

---

## ✅ 第二步：将当前用户加入 `docker` 组（避免每次用 sudo）

```bash
sudo usermod -aG docker $USER
newgrp docker  # 刷新组权限（或重新登录终端）
```

---

## ✅ 第三步：创建项目目录和持久化数据目录

```bash
# 创建项目目录
mkdir -p ~/timescaledb/{data,init}

# data/ 用于 PostgreSQL 数据文件
# init/ 可选：用于初始化 SQL 脚本（如创建超表等）
```

---

## ✅ 第四步：编写 `docker-compose.yml` 文件

创建文件：`~/timescaledb/docker-compose.yml`

```yaml
# ~/timescaledb/docker-compose.yml
version: '3.8'

services:
  timescaledb:
    image: timescale/timescaledb:latest-pg16
    container_name: timescaledb
    restart: always  # 关键：确保容器随 Docker 启动而启动
    ports:
      - "5432:5432"
    environment:
      POSTGRES_PASSWORD: mysecretpassword
      POSTGRES_DB: timescale
    volumes:
      - ./data:/var/lib/postgresql/data          # 持久化数据库文件
      - ./init:/docker-entrypoint-initdb.d       # 可选：初始化脚本目录
    deploy:
      resources:
        limits:
          memory: 1G
          cpus: '0.5'
        reservations:
          memory: 512M
          cpus: '0.2'
```

> 🔒 **安全建议**：生产环境应使用 `.env` 文件或 Docker secrets 管理密码，此处为简化使用明文。

---

## ✅ 第五步：启用 Docker 服务开机自启（系统级）

Docker 服务默认已启用开机自启，但请确认：

```bash
# 确保 docker 服务开机启动
sudo systemctl enable docker.service
sudo systemctl enable containerd.service

# 检查状态
systemctl is-enabled docker
# 应返回 "enabled"
```

> ✅ 由于我们在 `docker-compose.yml` 中设置了 `restart: always`，只要 Docker 服务启动，容器就会自动运行。

---

## ✅ 第六步：启动服务并验证

```bash
# 进入项目目录
cd ~/timescaledb

# 启动容器
docker compose up -d

# 查看运行状态
docker compose ps
# 或
docker ps -f name=timescaledb
```

### 验证数据持久化：
```bash
ls ~/timescaledb/data
# 应看到 PostgreSQL 数据文件（如 base/, global/, pg_wal/ 等）
```

### 连接测试：
```bash
# 安装 psql 客户端（可选）
sudo apt install -y postgresql-client

# 连接数据库
psql -h localhost -p 5432 -U postgres -d timescale
# 密码：mysecretpassword
```

---

## ✅ 第七步：验证开机自启（重启后测试）

```bash
sudo reboot
```

重启后执行：
```bash
docker ps -f name=timescaledb
```
应看到容器正在运行。

---

## 📁 目录结构说明

```
~/timescaledb/
├── docker-compose.yml
├── data/                  # PostgreSQL 数据目录（持久化）
└── init/                  # 可选：放 .sql 或 .sh 初始化脚本
```

> 所有重要数据都在 `~/timescaledb/data`，可备份、迁移或挂载到其他位置（如 `/opt/timescaledb-data`）。

---

## 🧹 可选：使用系统级 systemd 服务（高级方案）

虽然 `restart: always` + Docker 开机自启已足够，但若需更精细控制，可创建 systemd 服务：

```bash
# 创建服务文件
sudo tee /etc/systemd/system/timescaledb-compose.service <<EOF
[Unit]
Description=TimescaleDB via Docker Compose
Requires=docker.service
After=docker.service

[Service]
WorkingDirectory=/home/$USER/timescaledb
ExecStart=/usr/bin/docker compose up
ExecStop=/usr/bin/docker compose down
TimeoutStartSec=0
Restart=always
User=$USER

[Install]
WantedBy=multi-user.target
EOF
```

然后：
```bash
sudo systemctl daemon-reexec
sudo systemctl enable timescaledb-compose.service
sudo systemctl start timescaledb-compose.service
```

> ⚠️ 通常不需要此步骤，除非你希望完全绕过 Docker 的 restart 策略。

---

## ✅ 总结

| 目标 | 实现方式 |
|------|--------|
| 使用 `apt` 安装 | `docker-ce`, `docker-compose-plugin` |
| 外部存储映射 | `volumes: ./data:/var/lib/postgresql/data` |
| 开机自动启动 | `restart: always` + `systemctl enable docker` |

cat docker-compose.yml
services:
  timescaledb:
    image: timescale/timescaledb:latest-pg16
    container_name: timescaledb
    restart: always  # 关键：确保容器随 Docker 启动而启动
    ports:
      - "5432:5432"
    environment:
      POSTGRES_PASSWORD: iamposgrestsdb
      POSTGRES_DB: timescale
    volumes:
      - ./data:/var/lib/postgresql/data          # 持久化数据库文件
      - ./init:/docker-entrypoint-initdb.d       # 可选：初始化脚本目录
    deploy:
      resources:
        limits:
          memory: 1G
          cpus: '0.5'
        reservations:
          memory: 512M
          cpus: '0.2'